In [10]:
import warnings
warnings.filterwarnings('ignore')

from astropy.table import QTable, join, vstack
import numpy as np
import os

from astropy.table import vstack

import sys
sys.path.append('./../')
import time

In [11]:
def get_concat(dir_endswith='catalogue', file_endswith='props_all.fits', output_dir='./', muscat_table_file=''):
    """ 
    Concatenate tables from all directories 

    Parameters
    ----------
    dir_endswith : str
        Directory name ends with this string, e.g. 'catalogue' or 'catalogue_isofix'
    filename : str
        File name ends with this string, e.g. 'props_all.fits' or 'props_all_association.fits'

    Returns
    -------
    table : astropy.table.Table
        Concatenated table
    """

    tables = []
    for root, dirs, files in os.walk(output_dir):
        if not root.endswith(dir_endswith):
            continue
        for file in files:
            if file.endswith(file_endswith):
                if 'dendro' in root:
                    continue

                filename = os.path.join(root, file)
                filemodtime = time.ctime(os.path.getmtime(filename))
                print(f"{filename} \t - {filemodtime}")
                tables += [QTable.read(os.path.join(root, file))]

    print('TOTAL NUMBER OF TABLES:', len(tables)) # Number of tables

    # Join tables
    table = vstack(tables)

    # Order by galaxy names and region IDs from MUSE
    gal_name = table['gal_name']
    table.remove_column('gal_name')
    table.add_column(gal_name, index=0)
    table.sort(['gal_name', 'region_ID'])

    # Mask out NaN values
    mask = np.isnan(np.array(table['HA6562_LUMINOSITY_HST']))
    table_masked = table[~mask]
    
    # TODO - update scripts to use v4 catalogue...  
    # REMOVE THE FOLLOWING
    #####################################
    ### Add MUSE catalogue properties ###
    # Replacing with v4 MUSE catalogue properties - for N2 and S3 TEMPS mainly...
    # Note that scripts used to make catalogue are using the v3 MUSE catalogue...

    table_muse = QTable.read(muscat_table_file)
    columns = table_muse.colnames
    for column in columns:
        table_muse.rename_column(column, column+'_MUSE')
    table_muse.rename_column('gal_name_MUSE', 'gal_name')
    table_muse.rename_column('region_ID_MUSE', 'region_ID')
    table_muse.rename_column('Lum_HA6562_CORR_MUSE', 'HA6562_LUMINOSITY_MUSE')

    table_masked = join(table_masked, table_muse, keys=['gal_name', 'region_ID'])

    for colname in table_masked.colnames:
        if '_MUSE_1' in colname:
            table_masked.remove_column(colname)
        if '_MUSE_2' in colname:
            table_masked.rename_column(colname, colname.replace('_MUSE_2', '_MUSE'))
        if 't_N2' in colname:
            table_masked.remove_column(colname)
        if 't_S3' in colname:
            table_masked.remove_column(colname)
    #####################################

    return table_masked

In [12]:
# Define names and filenames...

output_dir = '/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/'
muscat_table_file = '/Users/abarnes/Library/CloudStorage/Dropbox/Data/Extragalactic/catalogues/Nebulae_catalogue_v4/Nebulae_catalogue_v4.fits'

print(output_dir)
print(muscat_table_file)

/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/
/Users/abarnes/Library/CloudStorage/Dropbox/Data/Extragalactic/catalogues/Nebulae_catalogue_v4/Nebulae_catalogue_v4.fits


In [13]:
# For all galaxies - WITHOUT stellar properties...
table_WITHOUT_associations = get_concat('catalogue_fixediso_10pc', 'props_all.fits', output_dir, muscat_table_file)
table_WITHOUT_associations.write(output_dir+'/all/nebulae_catalogue_fixediso_10pc.fits', overwrite=True)

/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc3627/catalogue_fixediso_10pc/props_all.fits 	 - Mon May 26 11:24:45 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc4254/catalogue_fixediso_10pc/props_all.fits 	 - Mon May 26 11:28:07 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc1512/catalogue_fixediso_10pc/props_all.fits 	 - Mon May 26 11:13:39 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc1433/catalogue_fixediso_10pc/props_all.fits 	 - Mon May 26 11:11:12 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc5068/catalogue_fixediso_10pc/props_all.fits 	 - Mon May 26 11:39:17 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc4321/catalogue_fixediso_10pc/props_all.fits 	 - Mon May 26 11:34:20 202

In [14]:
# For all galaxies - WITH stellar properties...
table_WITH_associations = get_concat('catalogue_fixediso_10pc', 'props_all_association.fits', output_dir, muscat_table_file)
table_WITH_associations.write(output_dir+'/all/nebulae_catalogue_wassociations_fixediso_10pc.fits', overwrite=True)

/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc3627/catalogue_fixediso_10pc/props_all_association.fits 	 - Mon May 26 11:25:32 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc4254/catalogue_fixediso_10pc/props_all_association.fits 	 - Mon May 26 11:29:25 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc1512/catalogue_fixediso_10pc/props_all_association.fits 	 - Mon May 26 11:14:14 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc1433/catalogue_fixediso_10pc/props_all_association.fits 	 - Mon May 26 11:11:36 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc5068/catalogue_fixediso_10pc/props_all_association.fits 	 - Mon May 26 11:39:50 2025
/Users/abarnes/Library/CloudStorage/Dropbox/Projects/Extragalactic/hstha_nebulae_catalogue/ngc4321/catalo

In [15]:
table_WITH_associations

gal_name,region_ID,x_max,y_max,x_com,y_com,ra_max,dec_max,ra_com,dec_com,npix,HA6562_FLUX_HST,HA6562_FLUX_ERR_HST,area_exact,radius_circ,flux_max,flux_min,flux_mean,radius_circ_pc,x_mom,y_mom,ra_mom,dec_mom,area_ellipse,major_sigma,minor_sigma,mean_sigma,position_angle,mean_sigma_pc,complexity_score,complexity_score_leaves,complexity_rms,complexity_std,flag_edge_hst,EW_HA6562_raw_MUSE,EW_HA6562_raw_ERR_MUSE,EW_HB4861_raw_MUSE,EW_HB4861_raw_ERR_MUSE,EW_HA6562_fit_MUSE,EW_HA6562_fit_ERR_MUSE,EW_HB4861_fit_MUSE,EW_HB4861_fit_ERR_MUSE,EW_HA6562_bgcorr_MUSE,EW_HA6562_bgcorr_ERR_MUSE,EW_HB4861_bgcorr_MUSE,EW_HB4861_bgcorr_ERR_MUSE,HA6562_FLUX_CORR_HST,HA6562_FLUX_ERR_CORR_HST,HA6562_LUMINOSITY_HST,HA6562_LUMINOSITY_ERR_HST,region_circ_rad_pc_MUSE,flag_touch_hst,region_ID_association_v_08pc,ra_association_v_08pc,dec_association_v_08pc,area_association_v_08pc,rad_association_v_08pc,age_association_v_08pc,age_err_association_v_08pc,mass_association_v_08pc,mass_err_association_v_08pc,ebv_association_v_08pc,ebv_err_association_v_08pc,age_ave_association_v_08pc,mass_sum_association_v_08pc,no_associations_association_v_08pc,one_associations_association_v_08pc,multiple_associations_association_v_08pc,region_ID_association_v_16pc,ra_association_v_16pc,dec_association_v_16pc,area_association_v_16pc,rad_association_v_16pc,age_association_v_16pc,age_err_association_v_16pc,mass_association_v_16pc,mass_err_association_v_16pc,ebv_association_v_16pc,ebv_err_association_v_16pc,age_ave_association_v_16pc,mass_sum_association_v_16pc,no_associations_association_v_16pc,one_associations_association_v_16pc,multiple_associations_association_v_16pc,region_ID_association_v_32pc,ra_association_v_32pc,dec_association_v_32pc,area_association_v_32pc,rad_association_v_32pc,age_association_v_32pc,age_err_association_v_32pc,mass_association_v_32pc,mass_err_association_v_32pc,ebv_association_v_32pc,ebv_err_association_v_32pc,age_ave_association_v_32pc,mass_sum_association_v_32pc,no_associations_association_v_32pc,one_associations_association_v_32pc,multiple_associations_association_v_32pc,region_ID_association_v_64pc,ra_association_v_64pc,dec_association_v_64pc,area_association_v_64pc,rad_association_v_64pc,age_association_v_64pc,age_err_association_v_64pc,mass_association_v_64pc,mass_err_association_v_64pc,ebv_association_v_64pc,ebv_err_association_v_64pc,age_ave_association_v_64pc,mass_sum_association_v_64pc,no_associations_association_v_64pc,one_associations_association_v_64pc,multiple_associations_association_v_64pc,region_ID_association_nuv_08pc,ra_association_nuv_08pc,dec_association_nuv_08pc,area_association_nuv_08pc,rad_association_nuv_08pc,age_association_nuv_08pc,age_err_association_nuv_08pc,mass_association_nuv_08pc,mass_err_association_nuv_08pc,ebv_association_nuv_08pc,ebv_err_association_nuv_08pc,age_ave_association_nuv_08pc,mass_sum_association_nuv_08pc,no_associations_association_nuv_08pc,one_associations_association_nuv_08pc,multiple_associations_association_nuv_08pc,region_ID_association_nuv_16pc,ra_association_nuv_16pc,dec_association_nuv_16pc,area_association_nuv_16pc,rad_association_nuv_16pc,age_association_nuv_16pc,age_err_association_nuv_16pc,mass_association_nuv_16pc,mass_err_association_nuv_16pc,ebv_association_nuv_16pc,ebv_err_association_nuv_16pc,age_ave_association_nuv_16pc,mass_sum_association_nuv_16pc,no_associations_association_nuv_16pc,one_associations_association_nuv_16pc,multiple_associations_association_nuv_16pc,region_ID_association_nuv_32pc,ra_association_nuv_32pc,dec_association_nuv_32pc,area_association_nuv_32pc,rad_association_nuv_32pc,age_association_nuv_32pc,age_err_association_nuv_32pc,mass_association_nuv_32pc,mass_err_association_nuv_32pc,ebv_association_nuv_32pc,ebv_err_association_nuv_32pc,age_ave_association_nuv_32pc,mass_sum_association_nuv_32pc,no_associations_association_nuv_32pc,one_associations_association_nuv_32pc,multiple_associations_association_nuv_32pc,region_ID_association_nuv_64pc,ra_association_nuv_64pc,dec

In [16]:
table_WITHOUT_associations

gal_name,region_ID,x_max,y_max,x_com,y_com,ra_max,dec_max,ra_com,dec_com,npix,HA6562_FLUX_HST,HA6562_FLUX_ERR_HST,area_exact,radius_circ,flux_max,flux_min,flux_mean,radius_circ_pc,x_mom,y_mom,ra_mom,dec_mom,area_ellipse,major_sigma,minor_sigma,mean_sigma,position_angle,mean_sigma_pc,complexity_score,complexity_score_leaves,complexity_rms,complexity_std,flag_edge_hst,EW_HA6562_raw_MUSE,EW_HA6562_raw_ERR_MUSE,EW_HB4861_raw_MUSE,EW_HB4861_raw_ERR_MUSE,EW_HA6562_fit_MUSE,EW_HA6562_fit_ERR_MUSE,EW_HB4861_fit_MUSE,EW_HB4861_fit_ERR_MUSE,EW_HA6562_bgcorr_MUSE,EW_HA6562_bgcorr_ERR_MUSE,EW_HB4861_bgcorr_MUSE,EW_HB4861_bgcorr_ERR_MUSE,HA6562_FLUX_CORR_HST,HA6562_FLUX_ERR_CORR_HST,HA6562_LUMINOSITY_HST,HA6562_LUMINOSITY_ERR_HST,region_circ_rad_pc_MUSE,flag_touch_hst,cen_x_MUSE,cen_y_MUSE,cen_ra_MUSE,cen_dec_MUSE,cen_x_geo_MUSE,cen_y_geo_MUSE,flag_edge_MUSE,flag_nearby_MUSE,flag_star_MUSE,deproj_dist_MUSE,deproj_phi_MUSE,region_area_MUSE,region_circ_rad_MUSE,HB4861_FLUX_MUSE,OIII5006_FLUX_MUSE,HA6562_FLUX_MUSE,NII6583_FLUX_MUSE,SII6716_FLUX_MUSE,SII6730_FLUX_MUSE,SIII9068_FLUX_MUSE,OI6300_FLUX_MUSE,NII5754_FLUX_MUSE,HEI5875_FLUX_MUSE,SIII6312_FLUX_MUSE,OI6363_FLUX_MUSE,OII7319_FLUX_MUSE,OII7330_FLUX_MUSE,HB4861_FLUX_ERR_MUSE,OIII5006_FLUX_ERR_MUSE,HA6562_FLUX_ERR_MUSE,NII6583_FLUX_ERR_MUSE,SII6716_FLUX_ERR_MUSE,SII6730_FLUX_ERR_MUSE,SIII9068_FLUX_ERR_MUSE,OI6300_FLUX_ERR_MUSE,NII5754_FLUX_ERR_MUSE,HEI5875_FLUX_ERR_MUSE,SIII6312_FLUX_ERR_MUSE,OI6363_FLUX_ERR_MUSE,OII7319_FLUX_ERR_MUSE,OII7330_FLUX_ERR_MUSE,HA6562_SIGMA_MUSE,NII6583_SIGMA_MUSE,OIII5006_SIGMA_MUSE,HA6562_VEL_MUSE,NII6583_VEL_MUSE,OIII5006_VEL_MUSE,HA6562_SIGMA_ERR_MUSE,NII6583_SIGMA_ERR_MUSE,OIII5006_SIGMA_ERR_MUSE,HA6562_VEL_ERR_MUSE,NII6583_VEL_ERR_MUSE,OIII5006_VEL_ERR_MUSE,HB4861_FLUX_CORR_MUSE,OIII5006_FLUX_CORR_MUSE,HA6562_FLUX_CORR_MUSE,NII6583_FLUX_CORR_MUSE,SII6716_FLUX_CORR_MUSE,SII6730_FLUX_CORR_MUSE,SIII9068_FLUX_CORR_MUSE,OI6300_FLUX_CORR_MUSE,NII5754_FLUX_CORR_MUSE,HEI5875_FLUX_CORR_MUSE,SIII6312_FLUX_CORR_MUSE,OI6363_FLUX_CORR_MUSE,OII7319_FLUX_CORR_MUSE,OII7330_FLUX_CORR_MUSE,HB4861_FLUX_CORR_ERR_MUSE,OIII5006_FLUX_CORR_ERR_MUSE,HA6562_FLUX_CORR_ERR_MUSE,NII6583_FLUX_CORR_ERR_MUSE,SII6716_FLUX_CORR_ERR_MUSE,SII6730_FLUX_CORR_ERR_MUSE,SIII9068_FLUX_CORR_ERR_MUSE,OI6300_FLUX_CORR_ERR_MUSE,NII5754_FLUX_CORR_ERR_MUSE,HEI5875_FLUX_CORR_ERR_MUSE,SIII6312_FLUX_CORR_ERR_MUSE,OI6363_FLUX_CORR_ERR_MUSE,OII7319_FLUX_CORR_ERR_MUSE,OII7330_FLUX_CORR_ERR_MUSE,EBV_MUSE,EBV_ERR_MUSE,met_scal_MUSE,met_scal_err_MUSE,logq_D91_MUSE,logq_D91_err_MUSE,BPT_NII_MUSE,BPT_SII_MUSE,BPT_OI_MUSE,LOG_SIGMA_SFR_CHAB_MUSE,LOG_SIGMA_STMASS_CHAB_MUSE,5thNN_dist_MUSE,NN_number_MUSE,Environment_MUSE,r_R25_MUSE,r_reff_MUSE,HA6562_LUMINOSITY_MUSE,Delta_met_scal_MUSE,pc_per_pixel_MUSE,EW_HA6562_MUSE,EW_HA6562_ERR_MUSE,EW_HB4861_MUSE,EW_HB4861_ERR_MUSE,HII_class_v2_MUSE,HII_class_v3_MUSE,FLAG_NII5754_MUSE,FLAG_SIII6312_MUSE,FLAG_OII7320_MUSE,FLAG_OII7330_MUSE,TEM_NII_MUSE,TEM_NII_ERR_MUSE,TEM_SIII_MUSE,TEM_SIII_ERR_MUSE
,,pix,pix,pix,pix,deg,deg,deg,deg,pix,erg / (s cm2),erg / (s cm2),arcsec2,arcsec,erg / (s cm2),erg / (s cm2),erg / (s cm2),pc,pix,pix,deg,deg,arcsec2,arcsec,arcsec,arcsec,deg,pc,,,,,,,,,,Angstrom,Angstrom,Angstrom,Angstrom,Angstrom,Angstrom,Angstrom,Angstrom,erg / (s cm2),erg / (s cm2),erg / s,erg / s,pc,,pix,pix,deg,deg,pix,pix,,,,arcsec,rad,,,erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),km / s,km / s,km / s,km / s,km / s,km / s,km / s,km / s,km / s,km / s,km / s,km / s,erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),erg / (s cm2),,,erg 